In [ ]:
#LIANA test for organoids it works with normalized adata
import liana as li
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

In [ ]:
#Homology pig to human conversion
resource = li.rs.select_resource('consensus')
resource.head()

map_df = li.rs.get_hcop_orthologs(url='https://ftp.ebi.ac.uk/pub/databases/genenames/hcop/human_pig_hcop_fifteen_column.txt.gz',
                                   columns=['human_symbol', 'pig_symbol'],
                                   min_evidence=3
                                   )
# rename the columns to source and target, respectively for the original organism and the target organism
map_df = map_df.rename(columns={'human_symbol':'source', 'pig_symbol':'target'})
map_df.tail()

#check conversion
pig = li.rs.translate_resource(resource,
                                 map_df=map_df,
                                 columns=['ligand', 'receptor'],
                                 replace=True,
                                 one_to_many=3
                                 )
pig

In [ ]:
# Run rank_aggregate for infering cell-cell interaction from all methods
li.mt.rank_aggregate(adata,
                     groupby='mergedcelltypes',
                     resource_name='pig',
                     resource=pig,
                     use_raw= False,
                     expr_prop=0.1,
                     verbose=True)

In [ ]:
#Plot and save results as dotplot
myplot = li.pl.dotplot(adata = adata,
              colour='magnitude_rank',
              size='specificity_rank',
              inverse_size=True,
              inverse_colour=True,
              source_labels=['Basal_epithelial_cells', 'Glycolysis_enriched_cells_1', 'Glycolysis_enriched_cells_2', 'Pancreatic_ductal_cells_1', 'Pancreatic_ductal_cells_2', 'Proliferating_cells'],
              target_labels=['WNT signaling enriched cells'],
              top_n=10,
              orderby='magnitude_rank',
              orderby_ascending=True,
              cmap ='Blues',
              figure_size=(12, 12)
             )

myplot.save('241114_PPDO_LIANA plot duct_to_WNT enriched_cells.pdf')